In [2]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

import spacy

import gc



nlp = spacy.load('en_core_web_lg')
from sklearn.metrics.pairwise import cosine_similarity

import itertools

# Get Data

## Changes from year to year

In [ ]:
changes = pd.read_csv("../../data/compustatChanges_all.csv").drop(columns = ['Unnamed: 0'])

changes.head()

In [ ]:
otherControls = pd.read_csv('../../data/companyData/otherControls.csv').\
    drop(columns = {'Unnamed: 0', 'fyearq'}).rename(columns = {'year_toMatchOn': 'year',
                                                              'fqtr': 'qtr'})


In [ ]:
otherControls.head()

In [ ]:
print(changes.shape)
changes = changes.merge(otherControls)
print(changes.shape)


industries = changes[['gvkey','famafrench']].drop_duplicates()

In [ ]:
'''changes.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changes.head()'''

In [2]:
changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv")

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Put in the calendar quarters and fiscal quarter data.

In [3]:
quarters = pd.read_csv("../../data/companyData/fiscalYears.csv")
quarters.head()

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,curcdq,datacqtr,datafqtr,costat,fyrc
0,1004,19980228,1997,3.0,5,INDL,C,D,STD,AIR,USD,1998Q1,1997Q3,A,5
1,1004,19980531,1997,4.0,5,INDL,C,D,STD,AIR,USD,1998Q2,1997Q4,A,5
2,1004,19980831,1998,1.0,5,INDL,C,D,STD,AIR,USD,1998Q3,1998Q1,A,5
3,1004,19981130,1998,2.0,5,INDL,C,D,STD,AIR,USD,1998Q4,1998Q2,A,5
4,1004,19990228,1998,3.0,5,INDL,C,D,STD,AIR,USD,1999Q1,1998Q3,A,5


In [4]:
len(quarters.gvkey.unique())

27703

In [5]:
sum((quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9))/quarters.shape[0]

0.8661478661478661

In [6]:
quarters = quarters[(quarters.fyr == 12) | 
   (quarters.fyr == 3) | 
   (quarters.fyr == 6) | 
   (quarters.fyr == 9)][['gvkey','datadate','datacqtr','datafqtr','fyr']].reset_index(drop = True)


In [7]:
quarters.head()

,gvkey,datadate,datacqtr,datafqtr,fyr
0,1010,19980331,1998Q1,1998Q1,12
1,1010,19980630,1998Q2,1998Q2,12
2,1010,19980930,1998Q3,1998Q3,12
3,1010,19981231,1998Q4,1998Q4,12
4,1010,19990331,1999Q1,1999Q1,12


Merge the quarter data into the change data, and make sure that the quarters that are used line up with the calendar quarters.

In [8]:
changesCal = changes[changes.gvkey.isin(quarters.gvkey.unique())]

changesCal = changesCal.merge(quarters)

print(changesCal.shape[0]/changes.shape[0])

1.0


In [9]:
changesCal.loc[~(changesCal.datacqtr.isna()), 'year'] = changesCal.datacqtr.str.slice(0,4)
changesCal.loc[~(changesCal.datacqtr.isna()), 'qtr']  = changesCal.datacqtr.str.slice(5,6)

changesCal['DATE'] = pd.to_datetime(changesCal['datadate'])

changesCal.loc[(changesCal.datacqtr.isna()), 'year'] = changesCal.DATE.dt.year
changesCal.loc[(changesCal.datacqtr.isna()), 'qtr']  = changesCal.DATE.dt.quarter

changesCal['year'] = changesCal.year.astype('int64')
changesCal['qtr']  = changesCal.qtr.astype('int64')

print(changesCal.shape,changesCal.head())

(645951, 44)    Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  gvkey  datadate  year  qtr  \
0           0             0               0   1010  20010331  2001    1   
1           1             1               1   1010  20010630  2001    2   
2           2             2               2   1010  20010930  2001    3   
3           3             3               3   1010  20011231  2001    4   
4           4             4               4   1010  20020331  2002    1   

          companyName curcdq  assets  ...  roa_lagged  famafrench  \
0  ACF INDUSTRIES INC    USD  3750.1  ...    0.005412        26.0   
1  ACF INDUSTRIES INC    USD  3701.7  ...    0.021410        26.0   
2  ACF INDUSTRIES INC    USD  3930.6  ...         NaN        26.0   
3  ACF INDUSTRIES INC    USD  3723.1  ...   -0.011506        26.0   
4  ACF INDUSTRIES INC    USD  3691.9  ...    0.019720        26.0   

   earliestYear  ageTercile  sizeTercile profitTercile datacqtr datafqtr fyr  \
0          1962           0          2.0 

In [18]:
changesCal.to_csv("../../data/companyData/compustatChanges_withControls.csv")
changesCal.head()

,Unnamed: 0,Unnamed: 0.1,gvkey,datadate,year,qtr,companyName,curcdq,assets,costGoodsSold,...,roa_lagged,famafrench,earliestYear,ageTercile,sizeTercile,profitTercile,datacqtr,datafqtr,fyr,DATE
0,0,0,1010,20010331,2001,1,ACF INDUSTRIES INC,USD,3750.1,56.3,...,0.005412,26.0,1962,0,2.0,1.0,2001Q1,2001Q1,12,1970-01-01 00:00:00.020010331
1,1,1,1010,20010630,2001,2,ACF INDUSTRIES INC,USD,3701.7,30.8,...,0.021410,26.0,1962,0,2.0,2.0,2001Q2,2001Q2,12,1970-01-01 00:00:00.020010630
2,2,2,1010,20010930,2001,3,ACF INDUSTRIES INC,USD,3930.6,37.0,...,NaN,26.0,1962,0,NaN,NaN,2001Q3,2001Q3,12,1970-01-01 00:00:00.020010930
3,3,3,1010,20011231,2001,4,ACF INDUSTRIES INC,USD,3723.1,30.0,...,-0.011506,26.0,1962,0,2.0,0.0,2001Q4,2001Q4,12,1970-01-01 00:00:00.020011231
4,4,4,1010,20020331,2002,1,ACF INDUSTRIES INC,USD,3691.9,26.1,...,0.019720,26.0,1962,0,2.0,2.0,2002Q1,2002Q1,12,1970-01-01 00:00:00.020020331


In [19]:
changesCal = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv")


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Compustat and ABI Linking

In [3]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns



hasMatch = gvKey_abiLinkingTable.gvkey.unique()

gvKey_abiLinkingTable.head()


,cstatCompanies,igCompanies,delete,gvkey,abi
0,asa gold and precious metals,asa gold precious metals,NaN,1062,402180222
1,adams diversified equity fd,adams diversified equity fund,NaN,1119,397759739
2,allen organ,allen organ,NaN,1283,400700704
3,american physicians svc gp,american physicians svc,NaN,1539,218548014
4,american science engineering,american science engineering,NaN,1554,441435880


---------------------------------

# Get all change data together
Get the linking table and merge the abi labels into the change df. 

Then, merge the location data into the change data and get as complete a record of companies as possible given the HQ data.

In [4]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
print(changes.shape, changes.head())


changesABI = changes.merge(gvKey_abiLinkingTable, on ='gvkey').drop(columns = {'state','city'})
print(changesABI.shape, changesABI.head())

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(645951, 43)    Unnamed: 0.1  Unnamed: 0.1.1  gvkey  datadate  year  qtr  \
0             0               0   1010  20010331  2001    1   
1             1               1   1010  20010630  2001    2   
2             2               2   1010  20010930  2001    3   
3             3               3   1010  20011231  2001    4   
4             4               4   1010  20020331  2002    1   

          companyName curcdq  assets  costGoodsSold  ...  roa_lagged  \
0  ACF INDUSTRIES INC    USD  3750.1           56.3  ...    0.005412   
1  ACF INDUSTRIES INC    USD  3701.7           30.8  ...    0.021410   
2  ACF INDUSTRIES INC    USD  3930.6           37.0  ...         NaN   
3  ACF INDUSTRIES INC    USD  3723.1           30.0  ...   -0.011506   
4  ACF INDUSTRIES INC    USD  3691.9           26.1  ...    0.019720   

   famafrench  earliestYear  ageTercile sizeTercile profitTercile datacqtr  \
0        26.0          1962           0         2.0           1.0   2001Q1   
1        26.0      

Now merge in the hq information.

In [12]:
canadian = ['ON', 'AB','QC', 'BC', 'NS', 'NF', 'SK', 'MB', 'NB']
changes = changes[~(changes.state.isin(canadian)) & ~changes.state.isna()]

changes['addzip'] = changes.addzip.astype('str').str.slice(0,5)

changes.state.unique()

array(['MO', 'MN', 'NY', 'NJ', 'PA', 'TX', 'SC', 'AZ', 'UT', 'IL', 'MA',
       'WA', 'CT', 'FL', 'CA', 'MD', 'NC', 'AL', 'HI', 'OH', 'AR', 'GA',
       'CO', 'NV', 'KS', 'ID', 'WI', 'ME', 'WY', 'VA', 'OK', 'VT', 'DE',
       'IN', 'PR', 'IA', 'MI', 'LA', 'RI', 'NE', 'MT', 'SD', 'OR', 'DC',
       'WV', 'KY', 'TN', 'MS', 'ND', 'NH', 'NM', 'VI', 'AK', 'GU'],
      dtype=object)

In [13]:
changes.drop(columns = {'Unnamed: 0.1'}, inplace = True)

In [14]:
hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

igChanges = changesABI.merge(hq)
print(igChanges.shape, igChanges.head())


hq.head()

(241835, 56)    Unnamed: 0.1  Unnamed: 0.1.1  gvkey  datadate  year  qtr  \
0            14              14   1013  20091231  2009    4   
1            15              15   1013  20100331  2010    1   
2            16              16   1013  20100630  2010    2   
3            17              17   1013  20100930  2010    3   
4           121             121   1045  20030331  2003    1   

                   companyName curcdq   assets  costGoodsSold  ...  \
0   ADC TELECOMMUNICATIONS INC    USD   1336.1          157.9  ...   
1   ADC TELECOMMUNICATIONS INC    USD   1345.1          158.5  ...   
2   ADC TELECOMMUNICATIONS INC    USD   1415.0          176.2  ...   
3   ADC TELECOMMUNICATIONS INC    USD   1474.5          184.6  ...   
4  AMERICAN AIRLINES GROUP INC    USD  29086.0         3943.0  ...   

                      company  state          city         address_line_1  \
0  ADC TELECOMMUNICATIONS INC     MN  EDEN PRAIRIE    13625 TECHNOLOGY DR   
1  ADC TELECOMMUNICATIONS INC    

,abi,ticker,company,year,state,city,address_line_1,zipcode,latitude,longitude,parent_employee_size_code,location_employee_size_code,employeesAtLocation
0,7609,SODI,SOLITRON DEVICES INC,2003,FL,WEST PALM BEACH,3301 ELECTRONICS WAY # C,33407,26.74120,-80.06694,NaN,NaN,NaN
1,23077,NaN,JENNY LEE BAKERY,2003,PA,MC KEES ROCKS,620 ISLAND AVE,15136,40.47235,-80.06152,50.0,50.0,1.0
2,76547,NaN,MASTER PROTECTION CORP,2003,FL,FORT MYERS,12800 UNIVERSITY DR # 400,33907,26.55504,-81.88423,20.0,20.0,1.0
3,77743,NaN,NATIONAL TECHNICAL SYSTEMS INC,2003,CA,CALABASAS,24007 VENTURA BLVD # 200,91302,34.15562,-118.65163,10.0,10.0,1.0
4,89151,NaN,HILLTOP BASIC RESOURCES,2003,OH,CINCINNATI,1 W 4TH ST # 1100,45202,39.09982,-84.51297,20.0,20.0,1.0


In [14]:
igChanges.to_csv("../../data/companyData/igData.csv")

At this point, we have zip information in the following forms (from most to least examples):
    - changes: all compustat companies, from the compustat address system
    - igChanges: subset of compustat companies, from the ig merge
    - subset of compustat companies that have SC information and survived the ig merge
    
We could potentially look at the subset of compustat companies for which we have SC information, usign the compustat address system as well.

For now: follow similar trajectory as before but add in weather data for all cstat companies and all ig-merged companies.

First: pull all zips that are mentioned in changes and igChanges and use this to get the weather data.



In [15]:
changes = changes[(~changes.addzip.isna()) & (changes.addzip != 'nan')]
relevantZips = changes.addzip.astype('int64').append(igChanges.zipcode).unique()

In [16]:
len(relevantZips)

4538

In [17]:
# relevantZips = allCustomerData.zipcode.append(allSupplierData.zipcode).unique()
outfile =  '../../data/companyData/relevantZips.pkl'
with open(outfile, 'wb') as pickle_file:
    pkl.dump(relevantZips, pickle_file)

------------------------------------------------

# Stocks

In [10]:
with open('../../data/stockReturns.pkl', 'rb') as f:
    stonks = pkl.load(f)[['date','gvkey','RET']]

In [11]:
stonks.head()

,date,gvkey,RET
0,2000-01-03,1690.0,0.088754
1,2000-01-04,1690.0,-0.084310
2,2000-01-05,1690.0,0.014634
3,2000-01-06,1690.0,-0.086538
4,2000-01-07,1690.0,0.047368


--------------------

# Weather Data

In [16]:
averages = pd.read_csv("../../data/companyData/quarterlyStatsByZip.csv").\
    drop(columns = {"Unnamed: 0"}).rename(columns = {'ZIP': 'zipcode'})


averages['qtr'] = averages.quarter.str.slice(1,2).astype('float')
averages.drop(columns = {'quarter'}, inplace = True) 

print(len(averages.zipcode.unique()))

averages.head()

4456


,zipcode,quarterly_avg_precip,quarterly_median_precip,quarterly_variance_precip,quarterly_avg_temp,quarterly_median_temp,quarterly_variance_temp,qtr
0,1001,2.866177,0.0,48.053550,4.253855,3.9460,42.033604,1.0
1,1001,3.601853,0.0,75.610613,20.817859,21.2870,49.057586,2.0
2,1001,3.468813,0.0,88.529803,26.791997,27.3020,20.188445,3.0
3,1001,3.435141,0.0,74.436349,10.708047,10.7795,55.548389,4.0
4,1013,2.755833,0.0,47.911696,3.964976,3.6070,42.290685,1.0


In [24]:
# allWeather = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2010.2019.csv").\
allWeather = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019.csv").\
    drop(columns = {"Unnamed: 0"})

allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

In [25]:
changes.rename(columns = {'addzip': 'zipcode'}, inplace = True)
changes.drop(columns = {'cik',
     'datadate','costat', 'add1', 'add2', 'city', 'sic', 'state'}, inplace = True)

In [26]:
col = allWeather.pop("yearQtr")
allWeather.insert(0, col.name, col)

In [27]:
allWeather.head()

,yearQtr,zipcode,year,qtr,precip_annualquant_0.95,precip_annualquant_1xQtr,precip_annualquant_1xYr,precip_annualquant_1x5Qtrs,precip_annualquant_1x10Qtrs,precip_annualquant_1x5Yrs,...,temp5Days_zipquant_1x10Qtrs,temp5Days_zipquant_1x5Yrs,temp5Days_zipquant_1x10Yrs,temp5Days_zipQuarterquant_0.95,temp5Days_zipQuarterquant_1xQtr,temp5Days_zipQuarterquant_1xYr,temp5Days_zipQuarterquant_1x5Qtrs,temp5Days_zipQuarterquant_1x10Qtrs,temp5Days_zipQuarterquant_1x5Yrs,temp5Days_zipQuarterquant_1x10Yrs
0,2009.00,1001,2009,1.0,9,3,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
1,2009.25,1001,2009,2.0,15,8,2,0,0,0,...,0,0,0,2,1,0,0,0,0,0
2,2009.50,1001,2009,3.0,13,9,4,0,0,0,...,0,0,0,2,0,0,0,0,0,0
3,2009.75,1001,2009,4.0,14,7,1,0,0,0,...,0,0,0,3,1,0,0,0,0,0
4,2010.00,1001,2010,1.0,10,9,5,0,0,0,...,0,0,0,3,2,0,0,0,0,0


In [28]:
lag1 = allWeather.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[4:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)

    
lag2 = allWeather.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[4:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)


lag3 = allWeather.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[4:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)


lag4 = allWeather.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[4:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)


print(allWeather.shape)

(184492, 88)


In [29]:
allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

allWeather_withLags.year.value_counts()

2010    16772
2011    16772
2012    16772
2013    16772
2014    16772
2015    16772
2016    16772
2017    16772
2018    16772
2019    16772
Name: year, dtype: int64

In [30]:
allWeather_withLags.to_csv("../../data/companyData/allWeather_withLags.csv")

Now do the same for the industry-specific weather.

In [ ]:
# allWeather = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2010.2019.csv").\
allWeather_byInd = pd.read_csv("../../data/companyData/revised_allWeatherBins_2009to2019_byInd.csv").\
    drop(columns = {"Unnamed: 0"})
'''[['famafrench','zipcode','yearQuarter', 
                                    'temp_ffquant_0.95','temp_indQuarterquant_0.95',
                                   'temp5Days_ffquant_0.95', 'temp5Days_indQuarterquant_0.95',
                                   'precip_ffquant_0.95', 'precip_indQuarterquant_0.95',
                                   'precip5Days_ffquant_0.95', 'precip5Days_indQuarterquant_0.95']]
'''
allWeather_byInd['year'] = allWeather_byInd.yearQuarter.str.slice(0,4).astype('int64')
allWeather_byInd['qtr']  = allWeather_byInd.yearQuarter.str.slice(5,6).astype('int64')
allWeather_byInd['yearQtr'] = allWeather_byInd.year + (allWeather_byInd.qtr - 1)/4

allWeather_byInd = allWeather_byInd.astype({'year':       'category',
                         'qtr':        'category',
                         'zipcode':    'category',
                         'famafrench': 'category'})

changes['zipcode'] = changes['zipcode'].astype({'zipcode': 'int64'})

changes = changes.astype({'year':       'category',
                          'qtr':        'category',
                          'zipcode':    'category',
                          'famafrench': 'category'})

col = allWeather_byInd.pop("year")
allWeather_byInd.insert(0, col.name, col)

col = allWeather_byInd.pop("qtr")
allWeather_byInd.insert(0, col.name, col)


col = allWeather_byInd.pop("yearQtr")
allWeather_byInd.insert(0, col.name, col)

allWeather_byInd.drop(columns = {'yearQuarter'}, inplace = True)

print(allWeather_byInd.head())

In [39]:
lag1 = allWeather_byInd.copy()
lag1['yearQtr'] += 0.25
for colname in lag1.columns[5:]:
    lag1.rename(columns = {colname: 'lag1_' + colname}, inplace = True)
lag1.drop(columns = {'year','qtr'},inplace = True)
lag1 = lag1.astype({'yearQtr':       'category'})

    
lag2 = allWeather_byInd.copy()
lag2['yearQtr'] += 0.5
for colname in lag2.columns[5:]:
    lag2.rename(columns = {colname: 'lag2_' + colname}, inplace = True)
lag2.drop(columns = {'year','qtr'},inplace = True)
lag2 = lag2.astype({'yearQtr':       'category'})


lag3 = allWeather_byInd.copy()
lag3['yearQtr'] += 0.75
for colname in lag3.columns[5:]:
    lag3.rename(columns = {colname: 'lag3_' + colname}, inplace = True)
lag3.drop(columns = {'year','qtr'},inplace = True)
lag3 = lag3.astype({'yearQtr':       'category'})


lag4 = allWeather_byInd.copy()
lag4['yearQtr'] += 1
for colname in lag4.columns[5:]:
    lag4.rename(columns = {colname: 'lag4_' + colname}, inplace = True)
lag4.drop(columns = {'year','qtr'},inplace = True)
lag4 = lag4.astype({'yearQtr':       'category'})


allWeather_byInd = allWeather_byInd.astype({'yearQtr':       'category'})


print(allWeather_byInd.shape)


allWeather_byInd.head()


'''allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

allWeather_byInd_withLags.year.value_counts()
'''

(6540864, 61)


'allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)\n\nallWeather_byInd_withLags.year.value_counts()\n'

In [40]:
allWeather_byInd_withLags = allWeather_byInd.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

In [41]:
allWeather_byInd_withLags.shape

(5946240, 285)

In [42]:
allWeather_byInd_withLags.to_csv("../../data/companyData/allWeather_byInd_withLags.csv")

In [45]:
del allWeather_byInd
del lag1
del lag2
del lag3
del lag4
gc.collect()

NameError: name 'allWeather_byInd' is not defined

# Locations
Create a separate definition of weather based not on HQ but on employee-weighted establishment footprint.

In [17]:
allWeather_byInd_withLags = pd.read_csv("../../data/companyData/allWeather_byInd_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})
allWeather_withLags = pd.read_csv("../../data/companyData/allWeather_withLags.csv").\
    drop(columns = {"Unnamed: 0", 'yearQtr'})




In [36]:
averages.head()

,zipcode,quarterly_avg_precip,quarterly_median_precip,quarterly_variance_precip,quarterly_avg_temp,quarterly_median_temp,quarterly_variance_temp,qtr
0,1001,2.866177,0.0,48.053550,4.253855,3.9460,42.033604,1.0
1,1001,3.601853,0.0,75.610613,20.817859,21.2870,49.057586,2.0
2,1001,3.468813,0.0,88.529803,26.791997,27.3020,20.188445,3.0
3,1001,3.435141,0.0,74.436349,10.708047,10.7795,55.548389,4.0
4,1013,2.755833,0.0,47.911696,3.964976,3.6070,42.290685,1.0


In [ ]:
allWeather_byInd_withLags = allWeather_byInd_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})
allWeather_withLags       = allWeather_withLags.astype({'year':       'category',
                           'qtr':        'category',
                           'zipcode':    'category'})
averages                  = averages.astype({'qtr':        'category',
                           'zipcode':    'category'})

In [20]:
len(averages.zipcode.unique())

4456

In [19]:
len(fractions.zipcode.unique())

37200

In [5]:
fractions = pd.read_csv('../../data/companyData/fractionEmployees_byEstablishment.csv').\
    drop(columns = {"Unnamed: 0", 'latitude','longitude'}).rename(columns = {'archive_version_year': 'year',
                                                    'parent_number': 'abi'})

fractions['year']    = fractions.year.astype('int64')
fractions['zipcode'] = fractions.zipcode.astype('int64')
fractions.head()

gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

print(gvKey_abiLinkingTable.abi)

gvKey_abiLinkingTable.head()

fractions = fractions[['year','abi','zipcode','locationFracOfEmployees']].merge(gvKey_abiLinkingTable[['abi','gvkey']])

fractions = fractions.astype({'year':       'category',
                           'zipcode':    'category'})

fractions.head()

0       402180222
1       397759739
2       400700704
3       218548014
4       441435880
          ...    
8394    432764101
8395    416920671
8396    739118540
8397    227688843
8398    488766353
Name: abi, Length: 8399, dtype: int64


,year,abi,zipcode,locationFracOfEmployees,gvkey
0,2003,100537,24541,0.047704,3937
1,2003,100537,24540,0.572451,3937
2,2003,100537,27828,0.357782,3937
3,2003,100537,27804,0.017889,3937
4,2003,100537,27834,0.001789,3937


In [ ]:
fractionsWithWeather = fractions.merge(allWeather_withLags).merge(allWeather_byInd_withLags).merge(averages)
print(fractionsWithWeather.shape)
fractionsWithWeather.head()

In [6]:
establishmentZips = fractions.zipcode.unique()
len(establishmentZips)

37200

Create direct effects database. Merge weather to full cstat database.

In [47]:
# changes['zipcode']  = changes['zipcode'].astype('int64')
changesWithWeather = changes.merge(allWeather_withLags).merge(allWeather_byInd_withLags).merge(averages)
print(changes.shape,changesWithWeather.shape)

(487582, 33) (189833, 740)


In [48]:
changesWithWeather.to_csv("../../data/companyData/cstatWithWeather.csv")

Merge weather to the ig-cstat database.

In [49]:
igChangesWithWeather = igChanges.merge(allWeather_withLags).merge(allWeather_byInd_withLags).merge(averages)
igChangesWithWeather.shape

(131383, 762)

In [50]:
igChangesWithWeather.to_csv("../../data/companyData/igWithWeather.csv")

In [ ]:
igChangesWithWeather.head()

In [54]:
igChangesWithWeather['temp_zipquant_0.95'].describe()

count    131383.000000
mean         10.639710
std          14.299951
min           0.000000
25%           0.000000
50%           4.000000
75%          17.000000
max          79.000000
Name: temp_zipquant_0.95, dtype: float64

In [52]:
igChangesWithWeather['temp5Days_ffquant_0.95'].describe()

count    131383.000000
mean         10.449975
std           9.267852
min           0.000000
25%           3.000000
50%           8.000000
75%          17.000000
max          37.000000
Name: temp5Days_ffquant_0.95, dtype: float64

# Indirect
Introduce the SC Data.

In [1]:
# this does a little bit of a test on the reporting requirements. 
# number 

'''c_linksTest = pd.read_csv("../../data/companyData/compustatSCLinked.csv")[['srcdate','gvkey','cgvkey']]
c_linksTest['year'] = c_linksTest.srcdate.astype('str').str.slice(0,4).astype('int64')

bs = c_linksTest[c_linksTest.year < 2014]
print("Customers per supplier, 1978-2013 Pd: ", len(bs.cgvkey.unique())/len(bs.gvkey.unique()))

bs2 = c_linksTest[c_linksTest.year > 2010]
print("Customers per supplier, Recent Pd: ", len(bs2.cgvkey.unique())/len(bs2.gvkey.unique()))'''


'c_linksTest = pd.read_csv("../../data/companyData/compustatSCLinked.csv")[[\'srcdate\',\'gvkey\',\'cgvkey\']]\nc_linksTest[\'year\'] = c_linksTest.srcdate.astype(\'str\').str.slice(0,4).astype(\'int64\')\n\nbs = c_linksTest[c_linksTest.year < 2014]\nprint("Customers per supplier, 1978-2013 Pd: ", len(bs.cgvkey.unique())/len(bs.gvkey.unique()))\n\nbs2 = c_linksTest[c_linksTest.year > 2010]\nprint("Customers per supplier, Recent Pd: ", len(bs2.cgvkey.unique())/len(bs2.gvkey.unique()))'

In [2]:
c_links = pd.read_csv("../../data/companyData/compustatSCLinked.csv")

c_links['year'] = c_links.srcdate.astype('str').str.slice(0,4).astype('int64')

c_links = c_links[c_links.year > 1999][['year','gvkey','cgvkey','salecs']].\
    rename(columns = {'cgvkey': 'customer_gvkey','gvkey': 'supplier_gvkey'})


print(c_links.shape)

c_links.head()

industries.columns = ['customer_gvkey','customer_famafrench']

c_links = c_links.merge(industries)

industries.columns = ['supplier_gvkey','supplier_famafrench']

c_links = c_links.merge(industries)
print(c_links.head(), c_links.shape)


c_links.to_csv("../../data/companyData/c_links.csv")


sum(c_links.supplier_gvkey.isin(hasMatch) | c_links.customer_gvkey.isin(hasMatch))


NameError: name 'pd' is not defined

Now see if it's common to have one in and one out of the industries of interest. 

For now, let's keep all the different industry types.

We can always filter later if we need to.

In [ ]:
#########################
# merge in customer information
gvKey_abiLinkingTable.columns = customer_columns

print(c_links.shape)
c_linksMerge1 = c_links.merge(gvKey_abiLinkingTable, on ='customer_gvkey')
print(c_links.shape,c_linksMerge1.shape)



#########################
# and merge in supplier 
gvKey_abiLinkingTable.columns = supplier_columns

print(c_links.shape)
c_linksMerge2 = c_linksMerge1.merge(gvKey_abiLinkingTable, on ='supplier_gvkey')
print(c_links.shape,c_linksMerge2.shape)

c_linksMerge2.to_csv("../../data/companyData/clinks_IG_selected.csv")

This is probably because: (1) companies are not in North America, or (2) companies are not in the physical goods industries we're interested in. We can verify this though: look at c_links where both the customer and supplier are in the dataset of interest.

In [ ]:
chq     = pd.read_csv("../../data/chq.csv",dtype={'cstatZipcode': 'object'}).drop(columns = {'Unnamed: 0'})

c_linkTest = c_links[c_links.customer_gvkey.isin(chq.gvkey.unique()) & \
                     c_links.supplier_gvkey.isin(chq.gvkey.unique())]

print("Percent of firms with a match: ", c_linksMerge2.shape[0]/c_linkTest.shape[0])

It's entirely possible that we have too small of a sample from the 2010s alone. Let's just try it though and see how it goes.

First, make a sample with the companies on three years of either side of when it reports another customer.

In [ ]:
def makeOneEitherSide(df): 
    yrPlus1 = df.copy(); yrPlus1['year'] += 1
    # yrPlus2 = df.copy(); yrPlus2['year'] += 1
    # yrPlus3 = df.copy(); yrPlus3['year'] += 1
    
    yrMinus1 = df.copy(); yrMinus1['year'] -= 1
    # yrMinus2 = df.copy(); yrMinus2['year'] -= 1
    # yrMinus3 = df.copy(); yrMinus3['year'] -= 1
    
    all = pd.concat([yrPlus1,yrMinus1]) # pd.concat([yrPlus1,yrPlus2,yrPlus3,yrMinus1,yrMinus2,yrMinus3])
    
    return(all)

In [ ]:
scTableCustomers = c_linksMerge2.copy()[['year','customer_gvkey','customer_abi']].drop_duplicates()
scTableSuppliers = c_linksMerge2.copy()[['year','supplier_gvkey','supplier_abi']].drop_duplicates()

allCustomerData = makeOneEitherSide(scTableCustomers)
allCustomerData.columns = ['year','gvkey','abi']


allSupplierData = makeOneEitherSide(scTableSuppliers)
allSupplierData.columns = ['year','gvkey','abi']

allAbi = allCustomerData.abi.append(allSupplierData.abi).drop_duplicates()

hqsOnly = pd.read_csv("../../data/ig_uniqueHQs.csv").drop(columns = {'Unnamed: 0'})

hq = pd.read_csv("../../data/ig_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

hqRelevant = hq[hq.abi.isin(allAbi)]


allSupplierData = allSupplierData.merge(hqRelevant).drop_duplicates()
allCustomerData = allCustomerData.merge(hqRelevant).drop_duplicates()

print(allSupplierData.head())

allCustomerData.to_csv("../../data/companyData/allCustomerData.csv")
allSupplierData.to_csv("../../data/companyData/allSupplierData.csv")

## Find Customer and Supplier pairings and merge with change data
### Can pick up here

In [2]:
allSupplierData = pd.read_csv("../../data/companyData/allSupplierData.csv").drop(columns = ['Unnamed: 0'])
allCustomerData = pd.read_csv("../../data/companyData/allCustomerData.csv").drop(columns = ['Unnamed: 0'])

changes = pd.read_csv("../../data/companyData/compustatChanges_withControls.csv").drop(columns = ['Unnamed: 0'])
changes.head()
suppliers = changes.merge(allSupplierData[['year','gvkey','zipcode','employeesAtLocation']])
print(suppliers.shape)

customers = changes.merge(allCustomerData[['year','gvkey','zipcode','employeesAtLocation']])
print(customers.head())

print(allCustomerData.shape,allSupplierData.shape)

## Get first-hop SC data

In [9]:
c_links = pd.read_csv("../../data/companyData/clinks_IG_selected.csv").drop(columns = {'Unnamed: 0'})
c_links.head()

,year,supplier_gvkey,customer_gvkey,salecs,customer_famafrench,supplier_famafrench,customer_cstatCompanies,customer_igCompanies,customer_delete,customer_abi,supplier_cstatCompanies,supplier_igCompanies,supplier_delete,supplier_abi
0,2002,1013,2136,111.056,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
1,2004,1013,2136,104.312,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
2,2005,1013,2136,146.000,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
3,2006,1013,2136,205.000,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129
4,2007,1013,2136,236.000,33.0,37.0,verizonmmunications,verizonmmunications,NaN,7564776,adc telecommunications,adc telecommunications,NaN,7523129


In [ ]:
c_links['suppliers'] = 1
custExp = c_links[['year', 'customer_gvkey', 'salecs','suppliers']].groupby(['year','customer_gvkey']).sum().\
    reset_index().rename(columns = {'salecs': 'totalExp'})

custExp.head()



In [ ]:
print("Number of firms with no exp information and multiple suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 1))
print("Number of firms with no exp information and >5 suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 5))


Most of these firms have expenditure information. We can look at:
    - Expenditure-weighted (just do equal shares if no exp information)
    - Largest supplier

In [ ]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()
print(customerDB.shape)

customerDB.head()

In [ ]:
customersWithWeather = customers.merge(allWeather_withLags).merge(averages).merge(allWeather_byInd_withLags)
customersWithWeather.shape

suppliersWithWeather = suppliers.merge(allWeather_withLags).merge(averages).merge(allWeather_byInd_withLags)
suppliersWithWeather.shape

suppliersWithWeather.to_csv("../../data/companyData/suppliersWithWeather.csv")
customersWithWeather.to_csv("../../data/companyData/customersWithWeather.csv")

'''suppliersWithWeather = pd.read_csv("../../data/companyData/suppliersWithWeather.csv").drop(columns = {'Unnamed: 0'})
customersWithWeather = pd.read_csv("../../data/companyData/customersWithWeather.csv").drop(columns = {'Unnamed: 0'})'''

frames = [customersWithWeather, suppliersWithWeather]

allCompanies = pd.concat(frames).drop_duplicates()

print(allCompanies.shape)

allCompanies.to_csv("../../data/companyData/allCompaniesWithWeather.csv")

## Biggest Supplier
Focus on weather of biggest supplier.

First find the max by supplier. Add back in any rows with only 1 supplier.

In [ ]:
customerDB.shape

In [ ]:
# https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby
idx = customerDB.groupby(['year','gvkey']).salecs.\
    transform(max) == customerDB.salecs
largestSuppliers = customerDB[idx].reset_index(drop = True)
print(largestSuppliers.shape)

# find companies who only have one other supplier
singleSuppliers = customerDB[customerDB.suppliers == 1].reset_index(drop = True)
print(singleSuppliers.shape)

# find largest suppliers of different companies
largestSuppliers = largestSuppliers.append(singleSuppliers).drop_duplicates()
print(largestSuppliers.shape)



In [ ]:
customerDB[['year','gvkey']][customerDB.year > 2009].drop_duplicates().shape

In [ ]:
len(largestSuppliers.gvkey.unique())

In [ ]:
for column in suppliersWithWeather.columns:
    print(column)

In [ ]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x) |
                                                 ('gvkey' in x) |
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[3:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey'},inplace = True)    

print(suppliers_toMerge.columns)


'''suppliers_toMerge = suppliersWithWeather[['year','qtr','gvkey','tmax_quant_1.0','precip_quant_1.0']].\
    rename(columns = {'gvkey': 'supplier_gvkey',
                      'tmax_quant_1.0': 'supplier_tmax_quant_1.0',
                      'precip_quant_1.0': 'supplier_precip_quant_1.0'})'''

In [ ]:
customersWithWeather.shape

In [ ]:
len(set(supplierWtdAvgWeather.gvkey.unique()) - set(suppliers_toMerge.gvkey.unique()))


In [ ]:
largestSuppliers.head()

In [ ]:
largestSuppliersWithWeather = customersWithWeather.merge(largestSuppliers[['year', 'gvkey', 'supplier_gvkey']]).merge(suppliers_toMerge)
largestSuppliersWithWeather.shape

In [ ]:
largestSuppliersWithWeather.head()

In [ ]:
largestSuppliersWithWeather.to_csv("../../data/companyData/largestSuppliersWithWeather.csv")


In [ ]:
largestSuppliersWithWeather.suppliers

In [4]:
largestSuppliersWithWeather = pd.read_csv("../../data/companyData/largestSuppliersWithWeather.csv")
largestSuppliersWithWeather.head()

,Unnamed: 0,year,gvkey,supplier_gvkey,salecs,totalExp,suppliers,datadate,qtr,companyName,...,supplier_lag4_temp5Days_zipQuarterquant_1x5Qtrs,supplier_lag4_temp5Days_zipQuarterquant_1x10Qtrs,supplier_lag4_temp5Days_zipQuarterquant_1x5Yrs,supplier_lag4_temp5Days_zipQuarterquant_1x10Yrs,supplier_quarterly_avg_precip,supplier_quarterly_median_precip,supplier_quarterly_variance_precip,supplier_quarterly_avg_temp,supplier_quarterly_median_temp,supplier_quarterly_variance_temp
0,0,2010,9899,111880,865.425,4633.447,37,20100331,1.0,AT&T INC,...,0,0,0,0,2.249304,0.0,44.801073,8.004915,7.6360,66.589459
1,1,2010,9899,111880,865.425,4633.447,37,20100630,2.0,AT&T INC,...,0,0,0,0,3.593418,0.0,82.171479,24.101767,25.0880,41.752215
2,2,2010,9899,111880,865.425,4633.447,37,20100930,3.0,AT&T INC,...,0,0,0,0,2.951039,0.0,76.675933,29.597434,30.0360,19.818994
3,3,2010,9899,111880,865.425,4633.447,37,20101231,4.0,AT&T INC,...,0,0,0,0,2.765759,0.0,56.969919,13.007701,13.4325,77.199622
4,4,2011,9899,111880,921.541,3977.293,37,20110331,1.0,AT&T INC,...,0,0,0,0,2.249304,0.0,44.801073,8.004915,7.6360,66.589459


In [8]:
largestSuppliersWithWeather.columns[0:50]

Index(['Unnamed: 0', 'year', 'gvkey', 'supplier_gvkey', 'salecs', 'totalExp',
       'suppliers', 'datadate', 'qtr', 'companyName', 'curcdq', 'assets',
       'costGoodsSold', 'totalInv', 'netIncome', 'totalRevenue', 'cik',
       'costat', 'add1', 'add2', 'addzip', 'city', 'sic', 'state',
       'assetsLast', 'netIncomeLast', 'totalRevenueLast', 'costGoodsSoldLast',
       'totalInvLast', 'incomeChange', 'revenueChange', 'costChange',
       'inventoryChange', 'assetsPrev', 'assetsLagged', 'netIncomeLagged',
       'roa_lagged', 'famafrench', 'earliestYear', 'ageTercile', 'sizeTercile',
       'profitTercile', 'zipcode', 'employeesAtLocation', 'yearQtr',
       'precip_annualquant_0.95', 'precip_annualquant_1xQtr',
       'precip_annualquant_1xYr', 'precip_annualquant_1x5Qtrs',
       'precip_annualquant_1x10Qtrs'],
      dtype='object')

## Sales-Weighted Average
If a company doesn't have sales-specific information, then assume equal shares. This doesn't happen for too many of the companies, thankfully.

In [ ]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()

customerDB['salesWeight'] = customerDB.salecs/customerDB.totalExp

customerDB.fillna(1, inplace = True)

Now merge this with the supplier weather data, and use the sales weights to find a sales-weighted average of the weather conditions for the suppliers.

In [ ]:
relevantVars = [x for x in suppliersWithWeather.columns if (('year' in x) | 
                                                 ('qtr' in x) |
                                                 ('gvkey' in x) |
                                                 ('_' in x)) & 
                                                ~('roa_lagged' in x) & 
                                                ~('yearQtr' in x)]

suppliers_toMerge = suppliersWithWeather[relevantVars]


for colname in suppliers_toMerge.columns[3:]:
    suppliers_toMerge.rename(columns = {colname: 'supplier_' + colname}, inplace = True)

    
suppliers_toMerge.rename(columns = {'gvkey': 'supplier_gvkey'},inplace = True)    


In [ ]:
suppliers_toMerge.head()

For each of the supplier weather columns, multiply the variable by the fraction of sales attributable to that relationship.

In [ ]:
supplierWeather = customerDB[['year','gvkey','supplier_gvkey','salesWeight']].merge(suppliers_toMerge)

for col in supplierWeather.columns[5:]:
        supplierWeather[col]   = supplierWeather.salesWeight*supplierWeather[col]
        
        

supplierWeather.drop(columns = {'supplier_gvkey','salesWeight'}, inplace = True)


print(supplierWeather.head())



# [['year','qtr','gvkey','supplier_tmax_quant_1.0','supplier_precip_quant_1.0']]

In [ ]:
supplierWtdAvgWeather = supplierWeather.groupby(['year','qtr','gvkey']).sum().reset_index().drop_duplicates()

In [ ]:
supplierWtdAvgWeather.gvkey.unique()

Merge the supplier weighted average weather data with the customer data that has weather as well.

In [ ]:
customersWithWeather.head()

In [ ]:
wtdAvgSuppliers = customersWithWeather.merge(supplierWtdAvgWeather)

wtdAvgSuppliers.head()

In [ ]:
wtdAvgSuppliers.shape

In [ ]:
wtdAvgSuppliers.to_csv("../../data/companyData/wtdAvgSuppliers.csv")

In [ ]:
wtdAvgSuppliers.head()

In [ ]:
wtdAvgSuppliers.columns[wtdAvgSuppliers.columns.str.contains('Tercile')]